# Export Energy Consumption data from Prometheus
Prometheus instance: http://k3s.fritz.box/prometheus-lt. Sample period: 15 min

Metrics:
- kpc_ac_energy_to_grid_wh_total
- kpc_home_power_consumption_wh_total{source="grid"}
- kpc_home_power_consumption_wh_total{source="pv"}

In [18]:
import pandas as pd
from prometheus_api_client import PrometheusConnect
from datetime import datetime, timedelta

# Time range
time_range_days = 365
end_time = datetime.now()  # Current time
start_time = end_time - timedelta(days=time_range_days)  # last x days

df = pd.DataFrame(columns=['timestamp', 'power'])
df_idx = 0

# Connect to the Prometheus server
prom = PrometheusConnect(url="http://k3s.fritz.box/prometheus-lt", disable_ssl=True)

# time range needs batching in 100d steps (max 11k data points allowed)
for days_start in range(0, time_range_days, 100):
    start_time_batch = start_time + timedelta(days=days_start)
    days_end = days_start + min(100, time_range_days - days_start)
    end_time_batch = start_time + timedelta(days=days_end)

    # fetch data from prometheus, max 11k data points allowed (~100d with 15min resolution)
    kpc_ac_energy_to_grid_wh_total = prom.custom_query_range(
        query="max(kpc_ac_energy_to_grid_wh_total)",
        start_time=start_time_batch,
        end_time=end_time_batch,
        step="15m"
        )
    kpc_home_power_consumption_grid_wh_total = prom.custom_query_range(
        query='max(kpc_home_power_consumption_wh_total{source="grid"})',
        start_time=start_time_batch,
        end_time=end_time_batch,
        step="15m"
    )
    kpc_home_power_consumption_pv_wh_total = prom.custom_query_range(
        query='max(kpc_home_power_consumption_wh_total{source="pv"})',
        start_time=start_time_batch,
        end_time=end_time_batch,
        step="15m"
    )

    # convert into power data, assumes same timestamps accross all queries

    kpc_ac_energy_to_grid_wh_total_values = kpc_ac_energy_to_grid_wh_total[0]['values']
    kpc_home_power_consumption_grid_wh_total_values = kpc_home_power_consumption_grid_wh_total[0]['values']
    kpc_home_power_consumption_pv_wh_total_values = kpc_home_power_consumption_pv_wh_total[0]['values']

    for i in range(1, len(kpc_ac_energy_to_grid_wh_total_values)):
        timestamp = datetime.fromtimestamp(kpc_ac_energy_to_grid_wh_total_values[i][0])
        delta_energy_to_grid = float(kpc_ac_energy_to_grid_wh_total_values[i][1]) - float(kpc_ac_energy_to_grid_wh_total_values[i-1][1])
        delta_energy_consumption_grid = float(kpc_home_power_consumption_grid_wh_total_values[i][1]) - float(kpc_home_power_consumption_grid_wh_total_values[i-1][1])
        power = (delta_energy_consumption_grid - delta_energy_to_grid) / 0.25 # 15min = 0.25h
        df.loc[df_idx] = {'timestamp': timestamp, 'power': power}
        df_idx += 1

df.to_csv("../../sample_data.csv")